In [1]:
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv
load_dotenv()
llm2 = ChatOpenAI()

messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm2.invoke(messages)
ai_msg


/Users/vinayak/AI/projects/mysql-chatbot-openai/mysql_chat/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


AIMessage(content="J'adore la programmation.")

In [54]:
import os
from dotenv import load_dotenv

load_dotenv()
db_user = "root"
db_password = "sak$103138"
db_host = "localhost"
db_name = "classicmodels"
from langchain_community.utilities.sql_database import SQLDatabase
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}")
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)


mysql
['customers', 'employees', 'offices', 'orderdetails', 'orders', 'payments', 'productlines', 'products']

CREATE TABLE customers (
	`customerNumber` INTEGER NOT NULL, 
	`customerName` VARCHAR(50) NOT NULL, 
	`contactLastName` VARCHAR(50) NOT NULL, 
	`contactFirstName` VARCHAR(50) NOT NULL, 
	phone VARCHAR(50) NOT NULL, 
	`addressLine1` VARCHAR(50) NOT NULL, 
	`addressLine2` VARCHAR(50), 
	city VARCHAR(50) NOT NULL, 
	state VARCHAR(50), 
	`postalCode` VARCHAR(15), 
	country VARCHAR(50) NOT NULL, 
	`salesRepEmployeeNumber` INTEGER, 
	`creditLimit` DECIMAL(10, 2), 
	PRIMARY KEY (`customerNumber`), 
	CONSTRAINT customers_ibfk_1 FOREIGN KEY(`salesRepEmployeeNumber`) REFERENCES employees (`employeeNumber`)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from customers table:
customerNumber	customerName	contactLastName	contactFirstName	phone	addressLine1	addressLine2	city	state	postalCode	country	salesRepEmployeeNumber	creditLimit
103	Atelier graphique	Schmit

In [55]:
from langchain.chains import create_sql_query_chain
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

question = "what is price of 1968 Ford Mustang"
encoded_query = question.encode('utf-8', errors='replace').decode('utf-8')
print(encoded_query)
#llm = ChatGroq(temperature=0)
llm = ChatOpenAI(temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": encoded_query})
# "what is price of `1968 Ford Mustang`"
print(query)


what is price of 1968 Ford Mustang
SELECT `buyPrice`, `MSRP`
FROM products
WHERE `productName` = '1968 Ford Mustang'
LIMIT 1;


In [56]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)


"[(Decimal('95.34'), Decimal('194.57'))]"

In [57]:
chain = generate_query | execute_query 
chain.invoke({"question":"How many orders are there"})

'[(326,)]'

In [58]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question and also display the answer in form of table in markdown.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)

print(chain.invoke({"question": "How many customers have an order count greater than 5"}))
print(chain.invoke({"question": "Display the customer tables with 5 rows"}))


There are 2 customers who have an order count greater than 5.

| customerCount |
|---------------|
|       1       |
|       1       |
The customer table with 5 rows is displayed below:

| CustomerNumber | CustomerName            | ContactLastName | ContactFirstName | Phone      | Address                  | City    | State | PostalCode | Country    | SalesRepEmployeeNumber | CreditLimit |
|----------------|-------------------------|-----------------|------------------|------------|--------------------------|---------|-------|------------|------------|------------------------|-------------|
| 103            | Atelier graphique       | Schmitt         | Carine           | 40.32.2555 | 54, rue Royale           | Nantes  |       | 44000      | France     | 1370                   | 21000.00    |
| 112            | Signal Gift Stores      | King            | Jean             | 7025551838 | 8489 Strong St.          | Las Vegas| NV    | 83030      | USA        | 1166                   | 71800.

## Few shot prompting technique 


In [59]:
examples = [
    {
        "input": "List all employees who were hired after January 1, 2003.",
        "query": "SELECT employeeNumber, lastName, firstName, hireDate \
                   FROM employees \
                   WHERE hireDate > '2003-01-01';"
    },
    {
        "input": "Find the total sales amount for each product.",
        "query": "SELECT productName, SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS totalSales \
                   FROM products \
                   JOIN orderdetails ON products.productCode = orderdetails.productCode \
                   GROUP BY productName;"
    },
    {
        "input": "List all customers located in Australia.",
        "query": "SELECT customerName, contactLastName, contactFirstName, phone \
                   FROM customers \
                   WHERE country = 'Australia';"
    },
    {
        "input": "Get the number of orders placed by each customer.",
        "query": "SELECT customerName, COUNT(*) AS totalOrders \
                   FROM customers \
                   JOIN orders ON customers.customerNumber = orders.customerNumber \
                   GROUP BY customerName;"
    },
    {
        "input": "Find the top 5 customers with the highest credit limits.",
        "query": "SELECT customerName, creditLimit \
                   FROM customers \
                   ORDER BY creditLimit DESC \
                   LIMIT 5;"
    }

    
]



In [60]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    # input_variables=["input","top_k"],
    input_variables=["input"],
)
print(few_shot_prompt.format(input="How many products are there?"))


Human: List all employees who were hired after January 1, 2003.
SQLQuery:
AI: SELECT employeeNumber, lastName, firstName, hireDate                    FROM employees                    WHERE hireDate > '2003-01-01';
Human: Find the total sales amount for each product.
SQLQuery:
AI: SELECT productName, SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS totalSales                    FROM products                    JOIN orderdetails ON products.productCode = orderdetails.productCode                    GROUP BY productName;
Human: List all customers located in Australia.
SQLQuery:
AI: SELECT customerName, contactLastName, contactFirstName, phone                    FROM customers                    WHERE country = 'Australia';
Human: Get the number of orders placed by each customer.
SQLQuery:
AI: SELECT customerName, COUNT(*) AS totalOrders                    FROM customers                    JOIN orders ON customers.customerNumber = orders.customerNumber                    GROUP

In [61]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    vectorstore,
    k=2,
    input_keys=["input"],
)
example_selector.select_examples({"input": "how many employees we have?"})
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="How many products are there?"))


Human: Find the total sales amount for each product.
SQLQuery:
AI: SELECT productName, SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS totalSales                    FROM products                    JOIN orderdetails ON products.productCode = orderdetails.productCode                    GROUP BY productName;
Human: Get the number of orders placed by each customer.
SQLQuery:
AI: SELECT customerName, COUNT(*) AS totalOrders                    FROM customers                    JOIN orders ON customers.customerNumber = orders.customerNumber                    GROUP BY customerName;


In [65]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info"))
generate_query = create_sql_query_chain(llm, db,final_prompt)
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
print(chain.invoke({"question": "How many csutomers with credit limit more than 50000"}))


System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: Find the total sales amount for each product.
SQLQuery:
AI: SELECT productName, SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS totalSales                    FROM products                    JOIN orderdetails ON products.productCode = orderdetails.productCode                    GROUP BY productName;
Human: Get the number of orders placed by each customer.
SQLQuery:
AI: SELECT customerName, COUNT(*) AS totalOrders                    FROM customers                    JOIN orders ON customers.customerNumber = orders.customerNumber                    GROUP BY customerName;
Human: How many products are there?
There are 85 customers with a credit limit greater than 50000.

| customersCount |
|--------------

In [66]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.csv")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)

Table Name:productlines
Table Description:Stores information about the different product lines offered by the company, including a unique name, textual description, HTML description, and image. Categorizes products into different lines.

Table Name:products
Table Description:Contains details of each product sold by the company, including code, name, product line, scale, vendor, description, stock quantity, buy price, and MSRP. Linked to the productlines table.

Table Name:offices
Table Description:Holds data on the company's sales offices, including office code, city, phone number, address, state, country, postal code, and territory. Each office is uniquely identified by its office code.

Table Name:employees
Table Description:Stores information about employees, including number, last name, first name, job title, contact info, and office code. Links to offices and maps organizational structure through the reportsTo attribute.

Table Name:customers
Table Description:Captures data on cus

In [68]:

table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "give me details of customer and their order count"})
tables

[Table(name='customers'), Table(name='orders')]

In [70]:

def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "give me details of customer and their order count"})

['customers', 'orders']

In [71]:
chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "How many cutomers with order count more than 5"})

'There are 2 customers with more than 5 orders.\n\n| customersWithMoreThan5Orders |\n|------------------------------|\n|              2               |'

In [72]:

chain.invoke({"question": "Can you list their names?"})

"Yes, the names of the customers from Australia are:\n| customerName                |\n|-----------------------------|\n| Australian Collectors, Co.  |\n| Anna's Decorations, Ltd     |\n| Souveniers And Things Co.   |\n| Australian Gift Network, Co |\n| Australian Collectables, Ltd|"

#### Adding Memory

In [29]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="How many products are there?",table_info="some table info",messages=[]))

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: Find the total sales amount for each product.
SQLQuery:
AI: SELECT productName, SUM(orderdetails.priceEach * orderdetails.quantityOrdered) AS totalSales                    FROM products                    JOIN orderdetails ON products.productCode = orderdetails.productCode                    GROUP BY productName;
Human: Get the number of orders placed by each customer.
SQLQuery:
AI: SELECT customerName, COUNT(*) AS totalOrders                    FROM customers                    JOIN orders ON customers.customerNumber = orders.customerNumber                    GROUP BY customerName;
Human: How many products are there?

In [30]:

from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)

In [31]:

question = "How many cutomers with order count more than 5"
response = chain.invoke({"question": question,"messages":history.messages})
response

'There are 2 customers with more than 5 orders.'

In [32]:
history.add_user_message(question)
history.add_ai_message(response)

In [33]:

history.messages

[HumanMessage(content='How many cutomers with order count more than 5'),
 AIMessage(content='There are 2 customers with more than 5 orders.')]

In [34]:
response = chain.invoke({"question": "Can you list there names?","messages":history.messages})
response

"The names of the customers with more than 5 orders are 'Mini Gifts Distributors Ltd.' and 'Euro+ Shopping Channel'."